In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from numpy.random import seed
seed(1)
# from tensorflow import set_random_seed
# set_random_seed(1)
import math
from numpy import arange
from sklearn.metrics import mean_squared_error 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from math import sqrt
import pandas as pd
from numpy.ma.core import ceil
from sklearn.model_selection import GridSearchCV
from copy import deepcopy
from sklearn.linear_model import ElasticNet
notebookpath = "/content/drive/MyDrive/Colab Notebooks/ADANIPORTS.csv"
# notebookpath = "/content/drive/MyDrive/Colab Notebooks/SMA STD Bol Upper,Lower and also some extra new features.csv"
# notebookpath ="/content/drive/MyDrive/Colab Notebooks/New Adani Datafrae with added features SMA,STD,Bollinger Upper,Bollinger Lower and removed useless features Series and Symbol.csv"
df = pd.read_csv(notebookpath);
# print(help(mean_squared_error))
df
# print(help(tf.keras.optimizers))
# print("-----------------------------------------------------")
# print(help(optimizers))

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.00,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.00,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.00,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.00,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.00,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123,0.2741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,2021-04-26,ADANIPORTS,EQ,725.35,733.00,739.65,728.90,729.2,730.75,733.25,9390549,6.885658e+14,116457.0,838079,0.0892
3318,2021-04-27,ADANIPORTS,EQ,730.75,735.00,757.50,727.35,748.6,749.15,747.67,20573107,1.538191e+15,236896.0,1779639,0.0865
3319,2021-04-28,ADANIPORTS,EQ,749.15,755.00,760.00,741.10,743.4,746.25,751.02,11156977,8.379106e+14,130847.0,1342353,0.1203
3320,2021-04-29,ADANIPORTS,EQ,746.25,753.20,765.85,743.40,746.4,746.75,753.06,13851910,1.043139e+15,153293.0,1304895,0.0942


In [ ]:
#preprocessing 
meantrades = df['Trades'].mean()
print(meantrades)
df['Trades'].fillna(value=meantrades, inplace=True)
 
# df = df.drop(['Volume'], axis = 1)
# df = df.drop(['Turnover'], axis = 1)
# df = df.drop(['Trades'], axis = 1)
# df = df.drop(['Deliverable Volume'], axis = 1)
# df = df.drop(['%Deliverble'], axis = 1)
# df = df.drop(['VWAP'], axis = 1)
# df = df.drop(['Last'], axis = 1)
# df = df.drop(['Low'], axis = 1)
df = df.drop(columns = ['Symbol','Series'   ], axis = 1)
# df = df.drop(['Prev Close'], axis = 1)

# print(df)
# normalize the dataset
lastcol = len(df.columns)
dfstr = df[df.columns[list(range(0,2))]]
# print(dfstr)
dfnum = df[df.columns[list(range(2,lastcol))]]
# dfnum = df[df.columns[list(range(8,8+1))]]
 
# print(type(dfnum))
cols = list(dfnum.columns)
 
 
dfnum = pd.DataFrame(dfnum, columns = cols)
  
# dfscaled = pd.concat([dfstr , dfnum], axis = 1, join = 'inner')
dfscaled = dfnum
 
print(dfscaled)


44922.58835504886
        Open     High     Low   Last   Close    VWAP    Volume      Turnover  \
0     770.00  1050.00  770.00  959.0  962.90  984.72  27294366  2.687719e+15   
1     984.00   990.00  874.00  885.0  893.90  941.38   4581338  4.312765e+14   
2     909.00   914.75  841.00  887.0  884.20  888.09   5124121  4.550658e+14   
3     890.00   958.00  890.00  929.0  921.55  929.17   4609762  4.283257e+14   
4     939.75   995.00  922.00  980.0  969.30  965.65   2977470  2.875200e+14   
...      ...      ...     ...    ...     ...     ...       ...           ...   
3317  733.00   739.65  728.90  729.2  730.75  733.25   9390549  6.885658e+14   
3318  735.00   757.50  727.35  748.6  749.15  747.67  20573107  1.538191e+15   
3319  755.00   760.00  741.10  743.4  746.25  751.02  11156977  8.379106e+14   
3320  753.20   765.85  743.40  746.4  746.75  753.06  13851910  1.043139e+15   
3321  739.00   759.45  724.50  726.4  730.05  743.35  12600934  9.366911e+14   

             Trades  

In [ ]:
# window 20, rmse corrected, data 1300:2300

le = len(dfscaled)
print(le)

window = [5]
# window = [15]

def transdata(dfscaled, win):
  df2 = deepcopy(dfscaled)
  col = df2.columns
  for i in range(2, win+1):
    for c in col:
      df2[c+str(i)] = df2[c].shift(-(i-1)) #stepback i-1 times
  
  df2['Close'+str(win+1)] = df2['Close'].shift(-(win)) #stepback i-1 times
  return df2
# print(help(Dense))
highcost = [152.42, 302.35, 359.43]
lowcost = [133.96, 251.55, 254.24]
ind = [(0,900,900,1100,1100,1300),(0,1900,1900, 2100, 2100, 2300),(0, 2800, 2800, 3000, 3000, 3200)]
for index in range(3):
  print('index'+str(index))
  for prevd in window:
    print('window '+str(prevd))
    #transforming data
    df2 = transdata(dfscaled, prevd)
    # print(df2)
    df2 = df2.dropna()
   
     
    # print('len(df2)')
    # print(len(df2))
    # print(df2)
 
    # df2['High'] = 
    # print(df2)

    #split
    dftr = deepcopy(df2.loc[ind[index][0]:ind[index][3]])
    # dftr = deepcopy(df2.loc[1300:2100])
    dftrY = dftr['Close'+str(prevd+1)].to_numpy()  
    dftrX = dftr.drop(['Close'+str(prevd+1)], axis = 1).to_numpy() 

  

    dftes = deepcopy(df2.loc[ind[index][4]:ind[index][5]])
    # dftes =  deepcopy(df2.loc[2100:2300])
    dftesY = dftes['Close'+str(prevd+1)].to_numpy()
    dftesX = dftes.drop(['Close'+str(prevd+1)], axis = 1).to_numpy()

    # dftrX,   dftesX = np.reshape(dftrX, (dftrX.shape[0], 1, dftrX.shape[1])),   np.reshape(dftesX, (dftesX.shape[0], 1, dftesX.shape[1]))
    # print(dftrX.shape)
    # print(dftesX.shape)
   
    
    # print(dftrX.shape)
    # print(dftrX.shape[1])
    
    regressor = ElasticNet()
    grid = dict()
    grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
    grid['l1_ratio'] = arange(0, 1, 0.1)
    search = GridSearchCV(regressor, grid, scoring='neg_root_mean_squared_error',   n_jobs=-1)
    leastvalerr = 1e9

    estimator = search.fit(dftrX, dftrY)
    tep = estimator.predict(dftesX)
    trp = estimator.predict(dftrX)
    import pickle
    pickle.dump(estimator, open('model_gridcv_on_regul_and_l1ratio.sav', 'wb'))
    # print(tep.shape)
   
     
    print("trainerror of best model")
    print(sqrt(mean_squared_error(trp, dftrY)))
    print("testerror of best model")
    print(sqrt(mean_squared_error(tep, dftesY)))
    high,mid,low = [],[],[]
    print(dftesY.shape)
    for i in range(len(dftesY)):
      # print('dftesY[i]')
      # print(dftesYin[i])
      if(dftesY[i] > highcost[index] ):
        high.append( (dftesY[i] - tep[i])**2)
      elif(dftesY[i] < lowcost[index]):
        low.append( (dftesY[i] - tep[i])**2)
      else:
        mid.append((dftesY[i] - tep[i])**2 )
    if(len(high) > 0):
      print('high')
      print(sqrt(sum(high)/len(high)))
    if(len(mid) > 0):
      print('mid')
      print(sqrt(sum(mid)/len(mid)))
    if(len(low) > 0):
      print('low')
      print(sqrt(sum(low)/len(low)))
     
        

3322
index0
window 5


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.524e+05, tolerance: 7.874e+03
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


trainerror of best model
26.46866640545304
testerror of best model
3.4230075575714225
(201,)
high
5.231892970300452
mid
4.882373206069844
low
2.6468332930012823
index1
window 5


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.641e+05, tolerance: 1.096e+04
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


trainerror of best model
19.52985635398089
testerror of best model
5.995287030047959
(201,)
high
6.620103587973184
mid
6.40367048659876
low
4.397426555168537
index2
window 5
trainerror of best model
16.879545319273344
testerror of best model
8.431062959262606
(201,)
high
5.836043004697632
mid
8.291187662664752
low
15.414581351013647


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.003e+05, tolerance: 1.128e+04
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [ ]:
#sanity checks
import pickle
pickle.dump(model[i].W50, open('best.sav', 'wb'))
